# Code to reproduce the figures found by Myles in Excel
Read in CO2 emissions per year, define a series of constants (not sure where these come from at the moment) and reproduce the graphs that Myles made in excel. The calculation of non-CO2 ERF changes prior to 2000, adding some complication to the code.
Next steps are to add error bars.


In [2]:
import numpy as np
import pandas as pd

# change the path  that points to the repository
# path = "C:\Users\galla\Documents\University Physics\Fourth Year Notes\MPhys_Project\2020_12_01InitialPaper\FlyingClimate-main\FlyingClimate-main\"
path = "~/git/FlyingClimate/"

In [3]:
# define constants to be used. 
# From sheet 'Time Series' in spreadsheet, 

alpha = 0.0       # BB6, TODO whats that?
AGWP100 = 91.7    # BB7
H = 100.0         # BB8
TCRE = 0.00045    # BB9

# effective radiative forcing (ERF) constants [mW/m^2/unit emission]
ERF = dict()                         # mean
ERFstd = dict()                      # uncertainty 1 standard deviation

# columns V-AD
ERF["O3short"] = 34.44               # ozone shortwave
ERF["CH4"] = -18.69                  # methane
ERF["O3long"] = -9.35                # ozone longwave
ERF["SWV"] = -2.80                   # stratospheric water vapour
ERF["netNOx"] = 5.46                 # nitrogen oxides (net)
ERF["BC"] = 100.67                   # black carbon
ERF["SO4"] = -19.91                  # sulphur oxides
ERF["H20"] = 0.0052                  # water vapour
ERF["contrail"] = 9.36e-10           # contrail and cirrus clouds [mW/m^2/km]

# same for uncertainty - 1 std
ERFstd["O3short"] = 9.9              # ozone shortwave
ERFstd["CH4"] = 6.9                  # methane
ERFstd["O3long"] = 3.4               # ozone longwave
ERFstd["SWV"] = 1.0                  # stratospheric water vapour
ERFstd["netNOx"] = 8.1               # nitrogen oxides (net)
ERFstd["BC"] = 165.5                 # black carbon
ERFstd["SO4"] = 16.0                 # sulphur oxides
ERFstd["H20"] = 0.0026               # water vapour
ERFstd["contrail"] = 0               # contrail and cirrus clouds [mW/m^2/km]

# other constants
Const = dict()
Const["distance_factor"] = 1.17
Const["total_to_civil"] = 0.95       # fraction
Const["EI_NOx"] = 15.14              # emission i? [g/kg fuel]
Const["EI_H2O"] = 1231.0             # emission i? [g/kg fuel]
Const["EI_BC"] = 0.03                # emission i? [g/kg fuel]
Const["EI_SO2"] = 1.2                # emission i? [g/kg fuel]

In [5]:
# Emission I? of nitrogen oxides
einox_years = np.arange(1976,2018)
EINOx = np.empty(len(einox_years))

# spot data
EINOx[einox_years == 1976] = 9.8
EINOx[einox_years == 1984] = 11.0
EINOx[einox_years == 1992] = 12.90
EINOx[einox_years == 2000] = 13.80
EINOx[einox_years == 2005] = 14.20
EINOx[einox_years == 2010] = 15.14

# keep constant in the future
EINOx[einox_years > 2010] = EINOx[einox_years == 2010]

# interpolate in between
for p in [(1976,1984),(1984,1992),(1992,2000),(2000,2005),(2005,2010)]:
    period = (einox_years >= p[0])*(einox_years <= p[1])
    slope = np.diff(EINOx[period][[-1,0]])[0]/ \
                    np.diff(einox_years[period][[-1,0]])[0]
    EINOx[period] = slope*(einox_years[period]-einox_years[period][0]) + \
                    EINOx[period][0]

In [12]:
np.vstack((einox_years,EINOx)).T

array([[1976.    ,    9.8   ],
       [1977.    ,    9.95  ],
       [1978.    ,   10.1   ],
       [1979.    ,   10.25  ],
       [1980.    ,   10.4   ],
       [1981.    ,   10.55  ],
       [1982.    ,   10.7   ],
       [1983.    ,   10.85  ],
       [1984.    ,   11.    ],
       [1985.    ,   11.2375],
       [1986.    ,   11.475 ],
       [1987.    ,   11.7125],
       [1988.    ,   11.95  ],
       [1989.    ,   12.1875],
       [1990.    ,   12.425 ],
       [1991.    ,   12.6625],
       [1992.    ,   12.9   ],
       [1993.    ,   13.0125],
       [1994.    ,   13.125 ],
       [1995.    ,   13.2375],
       [1996.    ,   13.35  ],
       [1997.    ,   13.4625],
       [1998.    ,   13.575 ],
       [1999.    ,   13.6875],
       [2000.    ,   13.8   ],
       [2001.    ,   13.88  ],
       [2002.    ,   13.96  ],
       [2003.    ,   14.04  ],
       [2004.    ,   14.12  ],
       [2005.    ,   14.2   ],
       [2006.    ,   14.388 ],
       [2007.    ,   14.576 ],
       [

In [16]:
# transient factor - what's that?
tf_years = np.arange(1980,2051)
transient_factor = np.empty(len(tf_years))

# spot data
transient_factor[tf_years == 2001] = 0.73
transient_factor[tf_years == 2005] = 0.75
transient_factor[tf_years == 2010] = 0.78
transient_factor[tf_years == 2018] = 0.79
transient_factor[tf_years == 2050] = 0.80

# keep constant into past & future
transient_factor[tf_years < 2001] = transient_factor[tf_years == 2001]

# interpolate in between
for p in [(2001,2005),(2005,2010),(2010,2018),(2018,2050)]:
    period = (tf_years >= p[0])*(tf_years <= p[1])
    slope = np.diff(transient_factor[period][[-1,0]])[0]/ \
                    np.diff(tf_years[period][[-1,0]])[0]
    transient_factor[period] = slope*(tf_years[period]-tf_years[period][0]) + \
                    transient_factor[period][0]

In [17]:
np.vstack((tf_years,transient_factor)).T

array([[1.980000e+03, 7.300000e-01],
       [1.981000e+03, 7.300000e-01],
       [1.982000e+03, 7.300000e-01],
       [1.983000e+03, 7.300000e-01],
       [1.984000e+03, 7.300000e-01],
       [1.985000e+03, 7.300000e-01],
       [1.986000e+03, 7.300000e-01],
       [1.987000e+03, 7.300000e-01],
       [1.988000e+03, 7.300000e-01],
       [1.989000e+03, 7.300000e-01],
       [1.990000e+03, 7.300000e-01],
       [1.991000e+03, 7.300000e-01],
       [1.992000e+03, 7.300000e-01],
       [1.993000e+03, 7.300000e-01],
       [1.994000e+03, 7.300000e-01],
       [1.995000e+03, 7.300000e-01],
       [1.996000e+03, 7.300000e-01],
       [1.997000e+03, 7.300000e-01],
       [1.998000e+03, 7.300000e-01],
       [1.999000e+03, 7.300000e-01],
       [2.000000e+03, 7.300000e-01],
       [2.001000e+03, 7.300000e-01],
       [2.002000e+03, 7.350000e-01],
       [2.003000e+03, 7.400000e-01],
       [2.004000e+03, 7.450000e-01],
       [2.005000e+03, 7.500000e-01],
       [2.006000e+03, 7.560000e-01],
 

In [11]:
### read in the data 

co2_forcing = pd.read_csv(path+"data/CO2.csv")

co2_emm = pd.read_csv(path+"data/CO2_emm_data.csv")
co2_emm

FileNotFoundError: [Errno 2] No such file or directory: '/Users/milan/git/FlyingClimate/data/CO2_emm_data.csv'

In [12]:
### remove null lines

co2 = co2_emm.dropna()
co2

NameError: name 'co2_emm' is not defined

### Pre 2000s
We will generate columns with the following headings for the dates pre 2000

'CO2 emm', 'non CO2 ERF', 'Cum CO2', 'Cum CO2 -fe', 'CO2 Warming', 'Non-CO2 warming'

In [73]:
### take only pre 2000 years of co2 data
co2_pre2000 = co2.iloc[:20]

In [74]:
### non co2 ERf
# first find contrail erf
contrail_erf = co2_pre2000['Contrail Distance/ km'] * contrail_ERF
## next add contrail erf and other non co2 erf's
non_co2_erf = contrail_erf + ((nonco2_erf_yr2000 - contrail_erf_yr2000) * co2_pre2000['Tg/yr'] / co2_emm_yr2000)
#non_co2_erf_df = pd.DataFrame(non_co2_erf, columns=['non CO2 ERF'])
co2_pre2000['non CO2 ERF'] = non_co2_erf
#co2_pre2000

C:\Users\galla\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [75]:
### Cumulative CO2

cum_co2 = []
for i in range(1, len(co2_pre2000) + 1) :
    co2_sum = co2_pre2000.iloc[:i, 1]
    cum_co2_i = co2_sum.sum(axis=0)/1000
    cum_co2.append(cum_co2_i)

        
co2_pre2000['Cum CO2'] = cum_co2
#co2_pre2000
    

C:\Users\galla\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [79]:
# Cumulative CO2-fe  - not sure what this is
cum_fe = (1 - alpha) * (H / AGWP100) * (co2_pre2000['non CO2 ERF'] - initial_non_co2_erf) + (alpha/AGWP100) * \
            initial_non_co2_erf * (co2_pre2000['Date'] - initial_date)
co2_pre2000['Cum CO2-fe'] = cum_fe
#co2_pre2000

C:\Users\galla\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [81]:
### CO2 Warming
co2_pre2000['CO2 Warming'] = co2_pre2000['Cum CO2'] * TCRE
#co2_pre2000

C:\Users\galla\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
### non CO2 Warming
co2_pre2000['non-CO2 Warming'] = co2_pre2000['Cum CO2-fe'] * TCRE
co2_pre2000

C:\Users\galla\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Tg/yr,Contrail Distance/ km,non CO2 ERF,Cum CO2,Cum CO2-fe,CO2 Warming,non-CO2 Warming
4,1980,406.8,1.090000e+10,14.148746,0.4068,-0.002160,0.000183,-9.720378e-07
5,1981,399.9,1.060000e+10,13.801009,0.8067,-0.381371,0.000363,-1.716170e-04
6,1982,405.1,1.070000e+10,13.945054,1.2118,-0.224288,0.000545,-1.009298e-04
7,1983,411.9,1.100000e+10,14.291821,1.6237,0.153865,0.000731,6.923917e-05
8,1984,439.4,1.180000e+10,15.307397,2.0631,1.261363,0.000928,5.676135e-04
9,1985,453.3,1.240000e+10,16.003840,2.5164,2.020843,0.001132,9.093795e-04
10,1986,478.7,1.340000e+10,17.186244,2.9951,3.310270,0.001348,1.489621e-03
11,1987,500.7,1.430000e+10,18.242065,3.4958,4.461656,0.001573,2.007745e-03
12,1988,525.0,1.520000e+10,19.320198,4.0208,5.637373,0.001809,2.536818e-03
13,1989,546.7,1.570000e+10,19.998708,4.5675,6.377298,0.002055,2.869784e-03
